### Import module

In [1]:
import numpy as np
from ultralytics import YOLO
import cv2

### Config Constant

In [2]:
accept_angle_threshold = 90
count = 0
in_up_position = False
in_down_position = False

In [3]:
def calculate_angle(shoulder_point, elbow_point, wrist_point):
    shoulder = np.array(shoulder_point)
    elbow = np.array(elbow_point)
    wrist = np.array(wrist_point)
    
    elbow_to_shoulder_vector = shoulder - elbow
    elbow_to_wrist_vector = wrist - elbow

    cos_angle = np.dot(elbow_to_shoulder_vector, elbow_to_wrist_vector) / (np.linalg.norm(elbow_to_shoulder_vector) * np.linalg.norm(elbow_to_wrist_vector))
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    angle = np.arccos(cos_angle)
    angle_degrees = np.degrees(angle)
    return angle_degrees

In [ ]:
model = YOLO('yolo11n-pose.pt')
# predict with the model
results = model.predict(source='90.png', show=True)

for result in results:
    if result.keypoints is not None:
        keypoints = result.keypoints.xy[0]
        if len(keypoints) > 9:
            left_elbow_point = keypoints[7]
            left_shoulder_point = keypoints[5]
            left_wrist_point = keypoints[9]
            left_angle_hand = calculate_angle(left_shoulder_point, left_elbow_point, left_wrist_point)
            print('Left Elbow Coordinates:', left_angle_hand)
            if left_angle_hand <= accept_angle_threshold:
                in_down_position = True
            else:
                in_up_position = True
        if len(keypoints) > 10:
            right_elbow_point = keypoints[8]
            right_shoulder_point = keypoints[6]
            right_wrist_point = keypoints[10]
            right_angle_hand = calculate_angle(right_shoulder_point, right_elbow_point, right_wrist_point)
            print('Right Elbow Coordinates:', right_angle_hand)
            if right_angle_hand <= accept_angle_threshold:
                in_down_position = True
            else:
                in_up_position = True
    if in_down_position and in_up_position:
        count += 1
        in_down_position = False
        in_up_position = False
        
print('Total Count:', count/2)
